In [ ]:
from DataPreprocessor import DataPreprocessor

# 创建预处理器
preprocessor = DataPreprocessor(
    aoi_geojson_path="../boundary/zhengzhou.geojson",
    local_cygnss_dir="/media/hl/Expansion/CYGNSS/CYGNSS_L1_V3.2/2021/7/",
    start_date="2021-07-01",
    end_date="2021-07-31",
    output_dir="/media/hl/Expansion/Results/rs_kalman_soil/zhengzhou_july2021/",
)

# 运行完整流程
results = preprocessor.run_full_pipeline()

# 或分步运行
preprocessor.initialize_gee()
preprocessor.load_aoi()
cygnss_data = preprocessor.read_cygnss_files()
era5_data = preprocessor.process_era5_data()
modis_data = preprocessor.process_modis_data()
imerg_data = preprocessor.process_imerg_data()
quality = preprocessor.check_data_quality()
preprocessor.export_data()